In [1]:
import numpy

In [2]:
# Open mesh in ply format
def open_ply(filepath):
    f=open(filepath,'r');
    i=0;
    ip=0;
    it=0;
    np=0;
    nt=0;
    for str in f:
        i+=1;
        arr=str.split(" ");
        if(arr[0]=="element" and arr[1]=="vertex"):
            np=int(arr[2]);
            p=numpy.zeros((np,3));
        elif(arr[0]=="element" and arr[1]=="face"):
            nt=int(arr[2]);
            t=numpy.zeros((nt,3));
        elif(i>=11 and i<11+np):
            p[ip,0]=float(arr[0]);
            p[ip,1]=float(arr[1]);
            p[ip,2]=float(arr[2]);
            ip+=1;
        elif(i>=11+np and i<11+np+nt):
            t[it,0]=int(arr[1]);
            t[it,1]=int(arr[2]);
            t[it,2]=int(arr[3]);
            it+=1;
    mesh={};
    mesh["np"]=np;
    mesh["nt"]=nt;
    mesh["p"]=p;
    mesh["t"]=t;
    return mesh;

In [3]:
pathDesktop = '/Users/ghfc/Desktop/'

# Load a txt file with the list of paired endpoints, such as each line has x y z of endpoint1 and x y z of endpoint2
fendpoints = pathDesktop+'P32_F16_endpoints_flipYX.txt'
endpoints = numpy.loadtxt(fendpoints)

In [4]:
endpoints[1]

array([ 40.5658,  18.4215,  10.6564])

Rotation

In [5]:
endpoints[:,0] *= -1
endpoints[:,2] *= -1

In [6]:
endpoints[1]

array([-40.5658,  18.4215, -10.6564])

In [7]:
print('xmin',endpoints[:,0].min(),'xmax',endpoints[:,0].max(),' ---- ',endpoints[:,0].max()-endpoints[:,0].min())
print('ymin',endpoints[:,1].min(),'ymax',endpoints[:,1].max(),' ---- ',endpoints[:,1].max()-endpoints[:,1].min())
print('zmin',endpoints[:,2].min(),'zmax',endpoints[:,2].max(),' ---- ',endpoints[:,2].max()-endpoints[:,2].min())

('xmin', -52.991100000000003, 'xmax', -24.088100000000001, ' ---- ', 28.903000000000002)
('ymin', 12.5162, 'ymax', 34.532299999999999, ' ---- ', 22.016100000000002)
('zmin', -24.222200000000001, 'zmax', -9.2386700000000008, ' ---- ', 14.98353)


In [8]:
fmesh = pathDesktop+'P32-F16_as_F01_402.ply'#'P32-F16_as_F01.ply'
mesh = open_ply(fmesh)
np=mesh["np"];
nt=mesh["nt"];
p=mesh["p"];
t=mesh["t"];

In [9]:
print('xmin',p[:,0].min(),'xmax',p[:,0].max(),' ---- ',p[:,0].max()-p[:,0].min())
print('ymin',p[:,1].min(),'ymax',p[:,1].max(),' ---- ',p[:,1].max()-p[:,1].min())
print('zmin',p[:,2].min(),'zmax',p[:,2].max(),' ---- ',p[:,2].max()-p[:,2].min())

('xmin', -13.602499999999999, 'xmax', 13.7883, ' ---- ', 27.390799999999999)
('ymin', -12.0951, 'ymax', 10.8005, ' ---- ', 22.895600000000002)
('zmin', -7.3936999999999999, 'zmax', 8.4204899999999991, ' ---- ', 15.81419)


In [10]:
xratio = (p[:,0].max()-p[:,0].min())/(endpoints[:,0].max()-endpoints[:,0].min())
yratio = (p[:,1].max()-p[:,1].min())/(endpoints[:,1].max()-endpoints[:,1].min())
zratio = (p[:,2].max()-p[:,2].min())/(endpoints[:,2].max()-endpoints[:,2].min())
print xratio,yratio,zratio

0.947680171608 1.03994803803 1.05543820448


In [27]:
#Scale down
scale = 0.88
endpoints[:,2] *= scale
endpoints[:,1] *= scale
endpoints[:,0] *= scale

Translation to mesh using centers

In [11]:
# Z centre fitting
mzmesh = (p[:,2].max()+p[:,2].min())/2
mzep = (endpoints[:,2].max()+endpoints[:,2].min())/2
print 'mzmesh',mzmesh
print 'mzep',mzmesh
zstep = mzmesh - mzep
print 'zstep',zstep

mzmesh 0.513395
mzep 0.513395
zstep 17.24383


In [8]:
# Z min fitting
zminmesh = p[:,2].min()
zminep = endpoints[:,2].min()
zstep = zminmesh - zminep
print 'zstep',zstep

16.819706


In [12]:
# Y centre fitting
mymesh = (p[:,1].max()+p[:,1].min())/2
myep = (endpoints[:,1].max()+endpoints[:,1].min())/2
print 'mymesh',mymesh
print 'myep',myep
ystep = mymesh - myep
print 'ystep',ystep

mymesh -0.6473
myep 23.52425
ystep -24.17155


In [10]:
# Y min fitting
yminmesh = p[:,1].min()
yminep = endpoints[:,1].min()
ystep = yminmesh - yminep
print 'ystep',ystep

-24.712601


In [10]:
# X centre fitting
mxmesh = (p[:,0].max()+p[:,0].min())/2
mxep = (endpoints[:,0].max()+endpoints[:,0].min())/2
print 'mxmesh',mxmesh
print 'mxep',mxep
xstep = mxmesh - mxep
print 'xstep',xstep

mxmesh -0.109
mxep -38.5396
xstep 38.4306


In [13]:
# X min fitting
xminmesh = p[:,0].min()
xminep = endpoints[:,0].min()
xstep = xminmesh - xminep
print 'xstep',xstep

xstep 39.3886


In [14]:
endpoints[:,2] += zstep
endpoints[:,1] += ystep
endpoints[:,0] += xstep

In [15]:
#Adjustments
endpoints[:,0] += 0.8 #0 #petit
endpoints[:,1] += 1.5 #1.2
endpoints[:,2] += 0.4 #0.5

In [16]:
from pandas import DataFrame
df = DataFrame(endpoints)
newfilename = 'P32_F16_endpoints_fit.txt'
df.to_csv(pathDesktop+newfilename, index=False, header=False,sep=' ')

In [23]:
print('xmin',endpoints[:,0].min(),'xmax',endpoints[:,0].max(),' ---- ',endpoints[:,0].max()-endpoints[:,0].min())
print('ymin',endpoints[:,1].min(),'ymax',endpoints[:,1].max(),' ---- ',endpoints[:,1].max()-endpoints[:,1].min())
print('zmin',endpoints[:,2].min(),'zmax',endpoints[:,2].max(),' ---- ',endpoints[:,2].max()-endpoints[:,2].min())

('xmin', -14.006261000000002, 'xmax', 14.896739, ' ---- ', 28.903000000000002)
('ymin', -11.701791499999997, 'ymax', 10.314308500000003, ' ---- ', 22.016100000000002)
('zmin', -6.9339274999999994, 'zmax', 8.0496025000000007, ' ---- ', 14.98353)


Formatting endpoints file for connectivity matrices script 

In [18]:
paired_ep = []
i = 0
while i <len(endpoints):
    
    paired_ep.append([endpoints[i][0],endpoints[i][1],endpoints[i][2],endpoints[i+1][0],endpoints[i+1][1],endpoints[i+1][2]])
    i +=2

In [19]:
from pandas import DataFrame
dfp = DataFrame(paired_ep)
newfilename = 'P32_F16_endpoints_fit_paired.txt'
dfp.to_csv(pathDesktop+newfilename, index=False, header=False, sep=' ')